1.首先用线性模型跑了一遍，发现FGA，BLK，TOV，AST四个参数的权重大一些<br\>
2.然后用SVR模型，kernel用rbf，和RidgeCV配合可以提高预测准确数<br\>
3.下一步做一下组合模型
- np.fabs()<br\>
- np.apply_along_axis()<br\>
- sklearn.model_selection.train_test_split()<br\>
- sklearn.model_selection.GridSearchCV()

In [26]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV
from sklearn import svm
from sklearn.model_selection import GridSearchCV

In [3]:
df = pd.read_excel("../数据收集/games.xlsx")
df.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22016,1610612749,MIL,Milwaukee Bucks,21601219,2017-04-12,MIL @ BOS,L,240,37,...,34,43,26,6,9,17,20,94,-18,1
1,22016,1610612738,BOS,Boston Celtics,21601219,2017-04-12,BOS vs. MIL,W,240,41,...,35,45,31,12,3,8,17,112,18,1
2,22016,1610612757,POR,Portland Trail Blazers,21601230,2017-04-12,POR vs. NOP,L,240,41,...,39,53,22,4,2,25,18,100,-3,1
3,22016,1610612740,NOP,New Orleans Pelicans,21601230,2017-04-12,NOP @ POR,W,240,40,...,31,42,20,13,3,11,17,103,3,1
4,22016,1610612746,LAC,LA Clippers,21601228,2017-04-12,LAC vs. SAC,W,240,43,...,32,43,29,4,6,6,15,115,20,1


In [4]:
df.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE'],
      dtype='object')

In [35]:
y = df['FGM'].get_values()
x = df[['FGA','BLK','TOV','AST']].get_values()
train_num = round(0.8*len(y))
x_scaler = StandardScaler().fit(x[:train_num]).transform(x)
#可以用sklearn.model_selection.train_test_split()完成训练集-测试集分离的过程
train_x,test_x = x_scaler[:train_num],x_scaler[train_num:]
train_y,test_y = y[:train_num],y[train_num:]
reg = RidgeCV([0.1,1,10]).fit(train_x,train_y)
#误差
err_ridge = reg.predict(test_x)-test_y
err_ridge_mean = np.fabs(err_ridge).mean()
correct_ridge = np.apply_along_axis(lambda x:[i for i,v in enumerate(x) if np.fabs(v)<0.5],0,err_ridge)
print('平均误差',err_ridge_mean,'\n正确个数',len(correct_ridge))

平均误差 2.76484336698 
正确个数 59


d:\programfiles\pyhton361\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [45]:
svr = svm.SVR()
clf = GridSearchCV(svr,{'C':[0.1,1,10]})
clf.fit(train_x,train_y)
err_svr = clf.predict(test_x)-test_y
err_svr_mean = np.fabs(err_svr).mean()
correct_svr = np.apply_along_axis(lambda x:[i for i,v in enumerate(x) if np.fabs(v)<0.5],0,err_svr)
print('平均误差',err_svr_mean,'\n正确个数',len(correct_svr))

平均误差 2.89510124798 
正确个数 59


In [46]:
clf.best_params_

{'C': 1}

In [51]:
len(set(np.concatenate([correct_svr,correct_ridge])))

87

In [48]:
correct_ridge

array([  0,   5,   8,  15,  16,  19,  26,  30,  32,  41,  43,  44,  58,
        70,  80,  84,  92, 108, 124, 126, 146, 153, 161, 162, 164, 166,
       167, 181, 200, 215, 219, 238, 250, 269, 270, 283, 293, 297, 305,
       322, 332, 368, 370, 374, 376, 386, 391, 419, 427, 430, 434, 439,
       445, 466, 469, 473, 474, 487, 491])